In [138]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 29 22:21:49 2020

@author: ghdbs
"""

from 

class DataGenerator(ArenaSplitter):
    """
    1. train을 27 : 3 : 1 로 나눈다.
    2. train2, train3 을 test 와 같은 모양으로 masking 한다
    3. stage1 에 쓰일 train, valid 를 한 파일로 만든다.
    """
    def __init__(self,):
        pass
    
    def split_train_test(self, train):
        train1, train2, train3 = self.split(train)
        train2_q = self.masking(train2)
        train3_q = self.masking(train3)
        return train1, train2, train3, train2_q, train3_q
    
    def masking(self, train, col, seed):
        #col을 불러온다.
        #인덱스 길이의 절반의 랜덤인덱스를 뽑는다
        #그 인덱스의 리스트는 제거한다.
        tot = len(train)
        song_only = 0.42
        song_tag = 0.39
        tag_only = 0.11
        title_only = 0.8
        for i in range(len(train)):
            row = train.iloc[i, ]
            list = row[col]
            
        
    def mask(self, train, col, seed):
        np.random.seed(seed)
        for i in range(len(train)):
            row = train.loc[i,col]
            no_mask = np.random.choice(row, size = (len(row)+1)//2, replace=False)
            train.at[i,col] = no_mask
        
    
    def split(self, train):
        length = len(train)
        train1 = train.iloc[:int(length * 27/31)]  
        train2 = train.iloc[int(length * 27/31):int(length * 30/31)]
        train3 = train.iloc[int(length * 30/31):]
        return train1, train2, train3

In [139]:
import pandas as pd
import numpy as np
train = pd.read_json("data/train.json").head(31)

In [1]:
DataGenerator().mask(train,"songs",2020)

NameError: name 'DataGenerator' is not defined

In [31]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 29 22:21:49 2020

@author: ghdbs
"""
import sys
sys.path.append('../')
from kakao_arena.split_data import ArenaSplitter
import numpy as np
import copy 
class DataGenerator(ArenaSplitter):
    """
    1. train을 27 : 3 : 1 로 나눈다.
    2. train2, train3 을 test 와 같은 모양으로 masking 한다
    3. stage1 에 쓰일 train, valid 를 한 파일로 만든다.
    """
    def __init__(self,):
        pass
    def _mask(self, playlists, mask_cols, del_cols):
        q_pl = copy.deepcopy(playlists) #playlist 깊은 복사 2개
        a_pl = copy.deepcopy(playlists)

        for i in range(len(playlists)):
            #플레이 리스트를 돌면서
            for del_col in del_cols:
                q_pl.at[i,del_col] = None
                #q_pl (플레이리스트) 의 i번째 행의 del_col 키값은 []로 만든다
                if del_col == 'songs':
                    #삭제하는 col이 songs이면 
                    a_pl.at[i,del_col] = a_pl.loc[i,del_col][:100]
                    #a_pl의 songs는 col은 100개까지로 줄인다
                elif del_col == 'tags':
                    #삭제하려는 col이 tags면
                    a_pl.at[i,del_col] = a_pl.loc[i,del_col][:10]
                    #a_pl의 tag는 10개까지로 줄인다

            for col in mask_cols:
                #마스크 하려는 컬럼을 돌면서
                mask_len = len(playlists.loc[i,col])
                #마스크 하려는 컬럼의 키값에 해당하는 밸류의 길이를 재고
                mask = np.full(mask_len, False)
                #mask len만큼 false로 채운 리스트를 두고
                mask[:mask_len//2] = True
                #절반은 True로 채우고
                np.random.shuffle(mask)
                #mask를 섞는다

                q_pl.at[i,col] = list(np.array(q_pl.loc[i,col])[mask])
                #q_pl 의 마스크하려는 컬럼은 mask True, False 에 해당하는 데이터만 뽑은 걸로 교체
                a_pl.at[i.col] = list(np.array(a_pl.loc[i,col])[np.invert(mask)])
                #a_pl의 마스크 하려는 컬럼은 mask의 반대 False, True에 해당하는 데이터만 뽑은걸로 교체
        return q_pl, a_pl
    
    def _mask_data(self, playlists):
        playlists = copy.deepcopy(playlists)
        tot = len(playlists)
        song_only = playlists[:int(tot * 0.42)]
        song_and_tags = playlists[int(tot * 0.42):int(tot * 0.81)]
        tags_only = playlists[int(tot * 0.81):int(tot * 0.92)]
        title_only = playlists[int(tot * 0.98):]

        print(f"Total: {len(playlists)}, "
              f"Song only: {len(song_only)}, "
              f"Song & Tags: {len(song_and_tags)}, "
              f"Tags only: {len(tags_only)}, "
              f"Title only: {len(title_only)}")
        
        song_q, song_a = self._mask(song_only, ['songs'], ['tags'])
        songtag_q, songtag_a = self._mask(song_and_tags, ['songs', 'tags'], [])
        tag_q, tag_a = self._mask(tags_only, ['tags'], ['songs'])
        title_q, title_a = self._mask(title_only, [], ['songs', 'tags'])
        q = song_q + songtag_q + tag_q + title_q
        a = song_a + songtag_a + tag_a + title_a

        shuffle_indices = np.arange(len(q))
        np.random.shuffle(shuffle_indices)

        q = list(np.array(q)[shuffle_indices])
        a = list(np.array(a)[shuffle_indices])

        return q, a  
    
    def _split(self, train):
        length = len(train)
        train1 = train.iloc[:int(length * 27/31)]  
        train2 = train.iloc[int(length * 27/31):int(length * 30/31)]
        train3 = train.iloc[int(length * 30/31):]
        return train1, train2, train3    

    def split_data(self, train):
        train1, train2, train3 = self._split(train)
        train2_q, train2_a = self._mask_data(train2)
        train3_q, train3_a = self._mask_data(train3)
        return train1, train2_q, train3_q, train2_a, train3_a

In [32]:
import pandas as pd
import numpy as np
train = pd.read_json("data/train.json").head(1000)

In [ ]:
%debug (DataGenerator().split_data(train))

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> s
--Call--
> <ipython-input-31-22e11c30ac4e>(18)__init__()
     16     3. stage1 에 쓰일 train, valid 를 한 파일로 만든다.
     17     """
---> 18     def __init__(self,):
     19         pass
     20     def _mask(self, playlists, mask_cols, del_cols):

ipdb> s
> <ipython-input-31-22e11c30ac4e>(19)__init__()
     17     """
     18     def __init__(self,):
---> 19         pass
     20     def _mask(self, playlists, mask_cols, del_cols):
     21         q_pl = copy.deepcopy(playlists) #playlist 깊은 복사 2개

ipdb> n
--Return--
None
> <ipython-input-31-22e11c30ac4e>(19)__init__()
     17     """
     18     def __init__(self,):
---> 19         pass
     20     def _mask(self, playlists, mask_cols, del_cols):
     21         q_pl = copy.deepcopy(playlists) #playlist 깊은 복사 2개

ipdb> s
--Call--
> <ipython-input-31-22e11c30ac4e>(91)split_data()
     89         return train1, train2, train3
     90 
---> 91     def s